In [1]:
import numpy as np
import tensorflow as tf
import random
import matplotlib.pyplot as plt
%matplotlib inline

/Users/adammarblestone/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
batch_size = 16
num_batches = 500

In [3]:
# Define a feedforward NN representing the sensory-motor system

input_vis = tf.placeholder(shape=[batch_size, 2], dtype=tf.float32, name = 'input_vis')
input_aud = tf.placeholder(shape=[batch_size, 2], dtype=tf.float32, name = 'input_aud')
input_total = tf.concat([input_vis, input_aud], axis = -1)

num_hidden_sensorymotor = 10
hidden_sensorymotor = tf.contrib.layers.fully_connected(input_total, num_hidden_sensorymotor, activation_fn = tf.nn.relu)
num_out_sensorymotor = 2
out_sensorymotor = tf.contrib.layers.fully_connected(hidden_sensorymotor, num_out_sensorymotor, activation_fn = tf.nn.relu)

In [4]:
# Just train the sensory-motor system to ignore audition
run_sensorymotor_test = False

if run_sensorymotor_test:
    loss = tf.reduce_mean(tf.norm(out_sensorymotor-input_vis))
    learning_rate = 0.01
    optimizer = tf.train.AdamOptimizer(learning_rate)
    train_op=optimizer.minimize(loss)
    
    sess = tf.Session()
    sess.run(tf.initialize_all_variables())
    
    num_inputs = batch_size*num_batches
    
    in_vis_list_unshaped = [[np.random.rand(), np.random.rand()] for k in range(num_inputs)] 
    in_vis_list = np.reshape(in_vis_list_unshaped, [num_batches,batch_size,2])
    
    in_aud_list_unshaped = [[np.random.rand(), np.random.rand()] for k in range(num_inputs)]
    in_aud_list = np.reshape(in_aud_list_unshaped, [num_batches,batch_size,2])

    losses = []
    for i in range(num_batches):
        in_vis_list_batch = in_vis_list[i]
        in_aud_list_batch = in_aud_list[i]
        l, it, os, _ =  sess.run([loss, input_total, out_sensorymotor, train_op], feed_dict = {input_vis:in_vis_list_batch, input_aud:in_aud_list_batch})
        losses.append(l)
        
    plt.figure()
    plt.plot(losses)
    plt.title("Loss function versus number of batches")
    plt.show()

In [5]:
# Define a feedforward NN representing the sensory-motor system with an ancillary rule label input

input_vis = tf.placeholder(shape=[batch_size, 2], dtype=tf.float32, name = 'input_vis')
input_aud = tf.placeholder(shape=[batch_size, 2], dtype=tf.float32, name = 'input_aud')
input_cue = tf.placeholder(shape=[batch_size, 1], dtype=tf.float32, name = 'input_cue')
input_total = tf.concat([input_vis, input_aud, input_cue], axis = -1)

num_hidden_sensorymotor = 10
hidden_sensorymotor = tf.contrib.layers.fully_connected(input_total, num_hidden_sensorymotor, activation_fn = tf.nn.relu)
num_out_sensorymotor = 2
out_sensorymotor = tf.contrib.layers.fully_connected(hidden_sensorymotor, num_out_sensorymotor, activation_fn = tf.nn.relu)

In [6]:
# Train the sensory-motor system to follow the given rule
run_sensorymotor_rule_test = False

if run_sensorymotor_rule_test:    
    
    num_inputs = batch_size*num_batches
    
    in_vis_list_unshaped = [[np.random.rand(), np.random.rand()] for k in range(num_inputs)] 
    in_vis_list = np.reshape(in_vis_list_unshaped, [num_batches,batch_size,2])
    
    in_aud_list_unshaped = [[np.random.rand(), np.random.rand()] for k in range(num_inputs)]
    in_aud_list = np.reshape(in_aud_list_unshaped, [num_batches,batch_size,2])
    
    cue_list_unshaped = [[np.random.rand()] for k in range(num_inputs)]
    in_cue_list = np.reshape(cue_list_unshaped, [num_batches,batch_size,1])
                
    learning_rate = 0.01
    optimizer = tf.train.AdamOptimizer(learning_rate)
    
    sess = tf.Session()
    
    losses = []
    for i in range(num_batches):
        in_vis_list_batch = in_vis_list[i]
        in_aud_list_batch = in_aud_list[i]
        in_cue_list_batch = in_cue_list[i]
        
        loss = 0
        for j in range(batch_size):
            if in_cue_list_batch[j] > 0.5:
                loss += tf.norm(tf.gather(out_sensorymotor-input_vis, [j]))
            else:
                loss += tf.norm(tf.gather(out_sensorymotor-input_aud, [j]))
        
        train_op=optimizer.minimize(loss)
        
        sess.run(tf.initialize_all_variables())
        l, it, os, _ =  sess.run([loss, input_total, out_sensorymotor, train_op], feed_dict = {input_vis:in_vis_list_batch, input_aud:in_aud_list_batch, input_cue:in_cue_list_batch})
        losses.append(l)
        print l
        
    plt.figure()
    plt.plot(losses)
    plt.title("Loss function versus number of batches")
    plt.show()

In [ ]:
# Now we will create a mapping from cues to rules
# The rule associated with each cue will change with time and a RNN will have to learn and remember the recent mapping
# while not hanging onto it too long as the mapping changes

# Define an RNN representing the PFC

if False: # Do we run this block

    num_units_PFC = 5

    PFC_cell = tf.contrib.rnn.LSTMBlockCell(num_units = num_units_PFC)
    PFC_state_previous = PFC_cell.zero_state(batch_size, tf.float32) # Initial state of PFC

    # This does one cycle of the RNN
    def PFC_step(input_data, network_state):
        with tf.variable_scope("PFC", reuse=False):
            return PFC_cell(inputs = input_data, state = network_state) 

    # Cue inputs

    num_timesteps = 5
    cue_timeseries = tf.placeholder(shape=[batch_size, num_timesteps, 1], dtype=tf.float32, name = 'cues_timeseries')

    for t in range(num_timesteps):

        current_cue = cue_timeseries[:,t]

        PFC_state = PFC_step(input_data = current_cue, network_state = PFC_state_previous)
        # The output from the PFC into the sensorymotor system will be what we were previously calling the cue variable
        # We'll now call it the rule
        PFC_output = tf.contrib.layers.fully_connected(PFC_state[0], 1, activation_fn = tf.nn.relu)

        PFC_state_previous = PFC_state[1]

    sess = tf.Session()

    ct = np.reshape([np.random.rand() for k in range(num_timesteps * batch_size * num_batches)], [num_batches, batch_size, num_timesteps, 1])

    sess.run(tf.initialize_all_variables())

    for b in range(num_batches):

            ct_in = ct[b, :, :]

            o = sess.run([PFC_output], feed_dict = {cue_timeseries:ct_in})

            print o

In [ ]:
# Now we will create a mapping from cues to rules
# The rule associated with each cue will change with time and a RNN will have to learn and remember the recent mapping
# while not hanging onto it too long as the mapping changes

# Define an RNN representing the PFC

num_units_PFC = 5

PFC_cell = tf.contrib.rnn.LSTMBlockCell(num_units = num_units_PFC)
PFC_state_previous = PFC_cell.zero_state(batch_size, tf.float32) # Initial state of PFC

# This does one cycle of the RNN
def PFC_step(input_data, network_state):
    with tf.variable_scope("PFC", reuse=False):
        return PFC_cell(inputs = input_data, state = network_state) 
    
# Cue inputs

num_timesteps = 200
cue_timeseries = tf.placeholder(shape=[batch_size, num_timesteps, 1], dtype=tf.float32, name = 'cues_timeseries')

input_vis = tf.placeholder(shape=[batch_size, num_timesteps, 2], dtype=tf.float32, name = 'input_vis')
input_aud = tf.placeholder(shape=[batch_size, num_timesteps, 2], dtype=tf.float32, name = 'input_aud')

def cue_to_rule_mapping_function(t, integer_input): # Define a changing mapping from cues to rules
    if t % 20 < 10:
        if integer_input == 0:
            return "vis"
        if integer_input == 1:
            return "vis"
        if integer_input == 2:
            return "aud"
        if integer_input == 3:
            return "aud"
    else:
        if integer_input == 0:
            return "aud"
        if integer_input == 1:
            return "aud"
        if integer_input == 2:
            return "vis"
        if integer_input == 3:
            return "vis"

loss = 0
        
for t in range(num_timesteps):

    current_cue = cue_timeseries[:,t]

    PFC_state = PFC_step(input_data = current_cue, network_state = PFC_state_previous)
    # The output from the PFC into the sensorymotor system will be what we were previously calling the cue variable
    # We'll now call it the rule
    PFC_output = tf.contrib.layers.fully_connected(PFC_state[0], 1, activation_fn = tf.nn.relu)
    
    input_cue_to_sm_sys = PFC_output
    input_total = tf.concat([input_vis[:,t], input_aud[:,t], input_cue_to_sm_sys], axis = -1)
    
    num_hidden_sensorymotor = 10
    hidden_sensorymotor = tf.contrib.layers.fully_connected(input_total, num_hidden_sensorymotor, activation_fn = tf.nn.relu)
    num_out_sensorymotor = 2
    out_sensorymotor = tf.contrib.layers.fully_connected(hidden_sensorymotor, num_out_sensorymotor, activation_fn = tf.nn.relu)
    
    PFC_state_previous = PFC_state[1]
    
    if cue_to_rule_mapping_function(t, current_cue) == "vis":
        loss += tf.norm(input_vis[:,t] - out_sensorymotor)
    else:
        loss += tf.norm(input_aud[:,t] - out_sensorymotor)
        
learning_rate = 0.01
optimizer = tf.train.AdamOptimizer(learning_rate)
train_op=optimizer.minimize(loss)

sess = tf.Session()

# Random cue
ct = np.reshape([np.random.randint(0,4) for k in range(num_timesteps * batch_size * num_batches)], [num_batches, batch_size, num_timesteps, 1])
    
in_vis_list_unshaped = [[np.random.rand(), np.random.rand()] for k in range(num_timesteps * batch_size * num_batches)] 
in_vis_list = np.reshape(in_vis_list_unshaped, [num_batches,batch_size,num_timesteps, 2])

in_aud_list_unshaped = [[np.random.rand(), np.random.rand()] for k in range(num_timesteps * batch_size * num_batches)]
in_aud_list = np.reshape(in_aud_list_unshaped, [num_batches,batch_size,num_timesteps,2])

sess.run(tf.initialize_all_variables())

for b in range(num_batches): # Just testing
        
        ct_in = ct[b, :, :]
        
        in_v = in_vis_list[b, :, :]
        
        in_a = in_aud_list[b, :, :]

        o, os, l, _ = sess.run([PFC_output, out_sensorymotor, loss, train_op], feed_dict = {cue_timeseries:ct_in, input_vis: in_v, input_aud:in_a})
        print "\n\n"
        print b
        print o
        print os
        print l




0
[[ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.00531004]
 [ 0.        ]
 [ 0.        ]
 [ 0.07549459]
 [ 0.        ]]
[[ 0.69727409  0.52413636]
 [ 0.83858919  0.88336813]
 [ 0.47693685  0.45418683]
 [ 0.87281489  1.02790582]
 [ 0.77472961  0.67257285]
 [ 0.32907724  0.56645417]
 [ 0.46733081  0.3841213 ]
 [ 0.83149916  0.583     ]
 [ 0.43225336  0.64385188]
 [ 0.81438959  0.87443852]
 [ 0.50024021  0.52057451]
 [ 0.74103582  0.88735783]
 [ 1.01782131  0.99016446]
 [ 0.8338415   0.85091388]
 [ 0.54584277  0.46372241]
 [ 0.51459724  0.67419487]]
561.138



1
[[ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]]
[[ 0.41640073  0.3734079 ]
 [ 0.39519984  0.51652795]
 [ 0.26514164  0.43990108]
 [ 0.5755676   0.59981292]
 [ 0.20356923  0.26564384]
 [ 0.48957857  0.57447374]
 [ 0.87115532  0.5840807 ]
 [ 0.8